## Bakersfield PM2.5 Prediction
 This notebook focuses on obtaining the best subset of the database for Bakersfield city in California to train a model for predicting PM2.5 concentration in the air.<br>
 
<img src="images/reference.png" alt="" style="width:450px;"> 
<img src="images/city.png" alt="" style="width:450px;">

## Merge the Data

In [ ]:
import pandas as pd

dfs = {}

for i in range(1999, 2025):
    dfs[f"df{i-1998}"] = pd.read_csv(f"data_raw/pm25_{i}.csv")

data = pd.concat(dfs.values(), ignore_index=True, verify_integrity=True)

data.to_csv('data_mod/data_merge.csv', index=False) 



## Check the Data

All the data in Bakersfield

In [ ]:
all((data['CBSA_NAME'].unique())=='Bakersfield, CA')

Split Bakersfield sites

In [ ]:
equal = data['Site Name'].nunique() == data['Site ID'].nunique()

if equal:
    print("IDs=Sites Names.")
else:
    print("IDs!=Sites Names.")


for i, (site_id, group) in enumerate(data.groupby('Site ID'), start=1):
    filename = f"data_mod/id_{i}.csv"
    group.to_csv(filename, index=False)


The site with the most data on Bakersfield

In [ ]:
num_rows_per_id = []

for site_id in range(1, 13):
    filename = f"data_mod/id_{site_id}.csv"
    df = pd.read_csv(filename)
    
    num_rows = len(df)
    num_rows_per_id.append(num_rows)

    print(f"The DataFrame for 'Site ID' {site_id} has {num_rows} rows.")


Check duplitcates

In [16]:
import pandas as pd

# Load the DataFrame from the CSV file
data = pd.read_csv("data_mod/id_3.csv")
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y', errors='coerce')

data_original = data.copy()

data_unique = data.drop_duplicates()

print(f'DataFrame original shape: {data_original.shape}')
print(f'data unique shape: {data_unique.shape}')

DataFrame original shape: (23321, 20)
data unique shape: (23321, 20)


## Let's start!!